#### MR_SUP__Wise_daily_FOS_ &CFT_Login & Logout_Report

Google Sheet Link - https://docs.google.com/spreadsheets/d/1Xfd6VDgIWuOTym0ClmbCYXuDW7OtGeqMg37ukItoXdw/edit?gid=0#gid=0

In [1]:
import pandas as pd
import pyodbc
import os
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

from datetime import datetime
import numpy as np

#### Setting up the GoogleSheet
Create a googlesheet with your userlogin , add python api editing rights </br>
define sheets name on google sheet and apply the open sheet methods

In [2]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:/Users/Administrator/LIS/GCPL SAMT AI/python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

# Open Sheet
sheet = client.open_by_key("1Xfd6VDgIWuOTym0ClmbCYXuDW7OtGeqMg37ukItoXdw") # Master Sheets ( Dimensions Sheet)
Data = sheet.worksheet("Data")
UserSheet = sheet.worksheet("User")

#### SQL Server Connections

In [3]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

#### SQL Query to fetch the data from SQL Server

In [4]:
query = """ 
WITH CTE AS 
(SELECT 
(TXT.UserName+'-'+ TXT.VISIT_DATE)AS UserVisitKey ,
TXT.Visit_Date,
TXT.UserName,
(TXT.VISIT_DATE +' '+TXT.CHEKINTIME) AS ChekinTime,
TXT.LISStoreCode,
(TXT.VISIT_DATE+'-'+TXT.LISStoreCode+'-'+TXT.UserName) AS StoreDateKey,
TXT.StoreName,TXT.Designation,
(TXT.VISIT_DATE +' '+TXT.CHECKOUTTIME ) AS Chekouttime,
TXT.FullName,
NoPermissionReason
FROM
(SELECT   FORMAT(CONVERT(DATETIME, SA.[VisitDate], 105), 'dd-MM-yyyy') AS Visit_Date,
SA.LISStoreCode,SA.StoreName, SA.UserName,SA.Designation,SA.FullName,
CONVERT(varchar, CAST([CheckIn-Time] AS TIME), 8) AS CHEKINTIME,
CONVERT(varchar, CAST([CheckOut-Time] AS TIME), 8) AS CHECKOUTTIME,
NoPermissionReason
FROM [PowerBI_LISV2].[DaburPharmaManagement].[ISPStoreAttendance_NormalExport] AS SA
LEFT JOIN [PowerBI_LISV2].[DaburPharmaManagement].UserMaster AS UM ON UM.UserName = SA.UserName
WHERE  [Month] = MONTH(GETDATE()-1) AND [Year] = YEAR(GETDATE()-1)
AND UM.Status =  'True' and  SA.UserName not like '%Star%'
)AS TXT
)
SELECT * FROM CTE;
"""

In [5]:
User = """
SELECT UM.EmployeeID,UM.UserName, UM.FullName,
UM.Designation,
UM.Region, UM.State,UM.City,UM.Status, UH.Supervisor
FROM [PowerBI_LISV2].[DaburPharmaManagement].UserMaster AS UM 
LEFT JOIN [PowerBI_LISV2].[DaburPharmaManagement].UserHierarchyListReport AS UH ON UH.ISP = UM.UserName
WHERE UM.Status = 'True' and UM.Designation in ('Merchandiser','Promoter') 
and  UM.UserName not like '%Star%' and  UM.UserName not like '%Test%' and  UM.UserName not like '%TEST%';

"""

# correct query matched 208 mr ( 22-08-2025)

#### Connections and extracting / reading data to Python Environment
Extracting all the required datasets for which we have created the SQL query , We can also do basic data cleaning while  writing SQL queries

In [6]:
conn = pyodbc.connect(conn_str)
df = pd.read_sql(query, conn)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_21460\3989963740.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [10]:
dfx= df[(df['UserName']== 'DBR-ISP114') & (df['Visit_Date']== '09-06-2025')]
dfx.head(10)

UserVisitKey  Visit_Date    UserName           ChekinTime  \
8435  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 18:05:09   
8831  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 14:08:42   
8844  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 14:02:31   
8855  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 13:58:18   
8878  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 13:48:39   
8893  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 13:42:50   
8902  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 13:40:20   
8927  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 13:32:05   
8937  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 13:29:26   
8947  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 13:23:29   

                        LISStoreCode  \
8435            DBRPH-DEL0DL11583514   
8831  DBRPH-0022007588DEL0DL00296961   
8844            DBRPH-DEL0DL00211962   
8855            DBRPH-DEL0DL11638595   
8878  DBRPH-0022013791DEL0DL00233733   
8893              DBRPH-DEL0DL729885   
8902             DBRPH-N_DL_00011276   
8927            DBRPH-DEL0DL00640852   
8937             DBRPH-N_DL_00011280   
8947            DBRPH-DEL0DL11534300   

                                           StoreDateKey  \
8435         09-06-2025-DBRPH-DEL0DL11583514-DBR-ISP114   
8831  09-06-2025-DBRPH-0022007588DEL0DL00296961-DBR-...   
8844         09-06-2025-DBRPH-DEL0DL00211962-DBR-ISP114   
8855         09-06-2025-DBRPH-DEL0DL11638595-DBR-ISP114   
8878  09-06-2025-DBRPH-0022013791DEL0DL00233733-DBR-...   
8893           09-06-2025-DBRPH-DEL0DL729885-DBR-ISP114   
8902          09-06-2025-DBRPH-N_DL_00011276-DBR-ISP114   
8927         09-06-2025-DBRPH-DEL0DL00640852-DBR-ISP114   
8937          09-06-2025-DBRPH-N_DL_00011280-DBR-ISP114   
8947         09-06-2025-DBRPH-DEL0DL11534300-DBR-ISP114   

                        StoreName   Designation          Chekouttime  \
8435       Shifa Diagnostic Cnter  Merchandiser  09-06-2025 18:28:07   
8831              SAHU SUPER MART  Merchandiser  09-06-2025 14:08:42   
8844  KHANPAAN DEPARTMENTAL STORE  Merchandiser  09-06-2025 14:02:31   
8855                  AHUJA STORE  Merchandiser  09-06-2025 13:58:18   
8878                 FRESHLY MART  Merchandiser  09-06-2025 13:56:33   
8893                 FRESHLY MART  Merchandiser  09-06-2025 13:48:19   
8902                 THE CHEMIEST  Merchandiser  09-06-2025 13:40:20   
8927              KITCHEN GALLERY  Merchandiser  09-06-2025 13:38:58   
8937              SANJEEVNI DRUGS  Merchandiser  09-06-2025 13:29:26   
8947                  PAL BROTHER  Merchandiser  09-06-2025 13:23:29   

       FullName NoPermissionReason  
8435  Dhirender               None  
8831  Dhirender        Stock Issue  
8844  Dhirender   Temporary Closed  
8855  Dhirender      Payment Issue  
8878  Dhirender               None  
8893  Dhirender               None  
8902  Dhirender      Payment Issue  
8927  Dhirender               None  
8937  Dhirender      Payment Issue  
8947  Dhirender        Stock Issue

In [11]:
User = pd.read_sql(User, conn)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_21460\1950467359.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  User = pd.read_sql(User, conn)


In [12]:
User.head()

EmployeeID    UserName      FullName   Designation Region        State  \
0  YM2024013510  DBR-ISP101      S.Lokesh  Merchandiser  South   Tamil Nadu   
1  YM2024013510  DBR-ISP101      S.Lokesh  Merchandiser  South   Tamil Nadu   
2  YM2024013510  DBR-ISP101      S.Lokesh  Merchandiser  South   Tamil Nadu   
3  YM2024013510  DBR-ISP101      S.Lokesh  Merchandiser  South   Tamil Nadu   
4  YM2022077171  DBR-ISP102  Tapas Naskar  Merchandiser   East  WEST BENGAL   

      City Status   Supervisor  
0    Arani   True  DBRPH-SUP-7  
1    Arani   True  DBRPH-SUP-7  
2    Arani   True  DBRPH-SUP-7  
3    Arani   True  DBRPH-SUP-7  
4  Kolkata   True  DBRPH-SUP-5

In [14]:
UserDataQuery ="""
SELECT 
distinct
UM.EmployeeID,UM.UserName, 
UM.FullName,
UM.Region, UM.State,UM.City,UM.Status, UH.Supervisor,UM2.FullName as SupervisorName
from [PowerBI_LISV2].[DaburPharmaManagement].UserMaster AS UM 
LEFT JOIN [PowerBI_LISV2].[DaburPharmaManagement].UserHierarchyListReport AS UH ON UH.ISP = UM.UserName
LEFT JOIN [PowerBI_LISV2].[DaburPharmaManagement].UserMaster AS UM2 ON UH.Supervisor = UM2.UserName
WHERE UM.Status = 'True' 
and  UM.UserName not like '%Star%' and  UM.UserName not like '%Test%' and  UM.UserName not like '%TEST%';

""" 

In [15]:
conn = pyodbc.connect(conn_str)
UserData = pd.read_sql(UserDataQuery, conn)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_21460\122418749.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  UserData = pd.read_sql(UserDataQuery, conn)


In [16]:
UserData2=UserData.drop(columns = ['FullName'])

In [17]:
UserData2.head() # do not add any columns while doing the join with the user table 

EmployeeID               UserName Region    State      City Status  \
0       None        AuditorDBRPH-01  North    Delhi     Delhi   True   
1       None        AuditorDBRPH-02  North    Delhi     Delhi   True   
2       None  DaburPharmaProduct-01  North    Delhi     Delhi   True   
3       None            DBRPH-AM-01  North  Haryana  Gurugram   True   
4       None                 LIS239  North    Delhi     Delhi   True   

  Supervisor SupervisorName  
0       None           None  
1       None           None  
2       None           None  
3       None           None  
4       None           None

In [276]:
dfy= df[(df['UserName']== 'DBR-ISP228') & (df['Visit_Date']== '03-06-2025')] 
dfy

UserVisitKey  Visit_Date    UserName           ChekinTime  \
1329  DBR-ISP228-03-06-2025  03-06-2025  DBR-ISP228  03-06-2025 19:48:02   
2348  DBR-ISP228-03-06-2025  03-06-2025  DBR-ISP228  03-06-2025 11:55:16   
2351  DBR-ISP228-03-06-2025  03-06-2025  DBR-ISP228  03-06-2025 11:54:52   
2405  DBR-ISP228-03-06-2025  03-06-2025  DBR-ISP228  03-06-2025 11:42:12   
2441  DBR-ISP228-03-06-2025  03-06-2025  DBR-ISP228  03-06-2025 11:30:50   
2518  DBR-ISP228-03-06-2025  03-06-2025  DBR-ISP228  03-06-2025 11:10:17   
2575  DBR-ISP228-03-06-2025  03-06-2025  DBR-ISP228  03-06-2025 10:52:22   
2710  DBR-ISP228-03-06-2025  03-06-2025  DBR-ISP228  03-06-2025 09:59:26   

                        LISStoreCode  \
1329             DBRPH-SLG0WB1915118   
2348            DBRPH-SLG0WB11572972   
2351            DBRPH-SLG0WB11571510   
2405              DBRPH-SLG0WB711743   
2441            DBRPH-SLG0WB11572948   
2518  DBRPH-0022014112SLG0WB00248352   
2575             DBRPH-SLG0WB1065376   
2710            DBRPH-SLG0WB11592898   

                                           StoreDateKey            StoreName  \
1329          03-06-2025-DBRPH-SLG0WB1915118-DBR-ISP228          BIDIT STORE   
2348         03-06-2025-DBRPH-SLG0WB11572972-DBR-ISP228   NEW GANESH BHANDER   
2351         03-06-2025-DBRPH-SLG0WB11571510-DBR-ISP228     BHARAMDEV PRASAD   
2405           03-06-2025-DBRPH-SLG0WB711743-DBR-ISP228    TRISHA ENTERPRISE   
2441         03-06-2025-DBRPH-SLG0WB11572948-DBR-ISP228          BHAGWAT SAH   
2518  03-06-2025-DBRPH-0022014112SLG0WB00248352-DBR-...       MAA ENTERPRISE   
2575          03-06-2025-DBRPH-SLG0WB1065376-DBR-ISP228  SOMNATH PICK N BILL   
2710         03-06-2025-DBRPH-SLG0WB11592898-DBR-ISP228          SWAPAN PAUL   

       Designation          Chekouttime        FullName  \
1329  Merchandiser  03-06-2025 20:11:34  Biswajit Modak   
2348  Merchandiser  03-06-2025 13:18:38  Biswajit Modak   
2351  Merchandiser  03-06-2025 11:54:52  Biswajit Modak   
2405  Merchandiser  03-06-2025 11:54:26  Biswajit Modak   
2441  Merchandiser  03-06-2025 11:37:10  Biswajit Modak   
2518  Merchandiser  03-06-2025 11:25:12  Biswajit Modak   
2575  Merchandiser  03-06-2025 11:09:33  Biswajit Modak   
2710  Merchandiser  03-06-2025 09:59:26  Biswajit Modak   

           NoPermissionReason  
1329                     None  
2348                     None  
2351  Permanent No Permission  
2405                     None  
2441                     None  
2518                     None  
2575                     None  
2710  Permanent No Permission

In [18]:
UserData2[UserData2['UserName']=='DBR-ISP114']

EmployeeID    UserName Region  State   City Status   Supervisor  \
24  YM202101947  DBR-ISP114  North  Delhi  Delhi   True  DBRPH-SUP-1   

   SupervisorName  
24  Gautam Mourya

#### Creating Model Logic

In [19]:
loginlogout = pd.merge(df,UserData2, how = 'left',on = 'UserName')

In [20]:
loginlogout['ChekinRank'] = loginlogout.groupby(['StoreDateKey'])['ChekinTime'].rank(method='dense', ascending=True).astype(int)


In [23]:
dfy= loginlogout[(loginlogout['UserName']== 'DBR-ISP114') & (loginlogout['Visit_Date']== '09-06-2025') & (loginlogout['LISStoreCode']=='DBRPH-0022007588DEL0DL00296961')]
# dfx = dfx.drop_duplicates()
dfy.head(10)

UserVisitKey  Visit_Date    UserName           ChekinTime  \
8831  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 14:08:42   

                        LISStoreCode  \
8831  DBRPH-0022007588DEL0DL00296961   

                                           StoreDateKey        StoreName  \
8831  09-06-2025-DBRPH-0022007588DEL0DL00296961-DBR-...  SAHU SUPER MART   

       Designation          Chekouttime   FullName NoPermissionReason  \
8831  Merchandiser  09-06-2025 14:08:42  Dhirender        Stock Issue   

       EmployeeID Region  State   City Status   Supervisor SupervisorName  \
8831  YM202101947  North  Delhi  Delhi   True  DBRPH-SUP-1  Gautam Mourya   

      ChekinRank  
8831           1

In [263]:
# loginlogout[loginlogout['LISStoreCode']=='DBRPH-BSR0WB00208343'].head()
# loginlogout2 = loginlogout2.sort_values('ChekinTime').drop_duplicates(subset='UserVisitKey', keep='first')

In [264]:
loginlogout2= loginlogout[loginlogout['ChekinRank']==1]

In [265]:
loginlogout2=loginlogout2.drop(columns = ['ChekinRank'])

In [266]:
loginlogout2.at[0, 'LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [267]:
loginlogout2['UserVisitKey'].count()

49549

In [268]:
# loginlogout2.head()

In [269]:
loginlogout2.to_csv(r'C:\Users\Administrator\LIS\Dabur\daburloginlogout.csv', index=False)

In [270]:
loginlogout2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49549 entries, 0 to 49780
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   UserVisitKey        49549 non-null  object
 1   Visit_Date          49549 non-null  object
 2   UserName            49549 non-null  object
 3   ChekinTime          49549 non-null  object
 4   LISStoreCode        49549 non-null  object
 5   StoreDateKey        49549 non-null  object
 6   StoreName           49549 non-null  object
 7   Designation         49549 non-null  object
 8   Chekouttime         49549 non-null  object
 9   FullName            49549 non-null  object
 10  NoPermissionReason  22521 non-null  object
 11  EmployeeID          49549 non-null  object
 12  Region              49549 non-null  object
 13  State               49549 non-null  object
 14  City                49549 non-null  object
 15  Status              49549 non-null  object
 16  Supervisor          49010 n

#### Updating the model output to the google sheet tabs 
As defined previously while creatng google sheets, We have already created the tabs- here we can specify the two things 1) erasing the previous data and updating the new one </br>
we can also define the specific cell or cell range on the we want to paste the output of this model

In [233]:
#Sending df on sheet
Data.batch_clear(["A1"])
set_with_dataframe(Data, loginlogout2)

UserSheet.batch_clear(["A1"])
set_with_dataframe(UserSheet, UserData)

In [238]:
print("Script Run finished")

Script Run Started


In [294]:
loginlogout2 = loginlogout2.drop_duplicates()